In [2]:
from bs4 import BeautifulSoup
import requests
# set and request url; extract source code
url = 'https://www.opentable.com/new-york-restaurant-listings'
html = requests.get(url)
html.text[:500]
' <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/> <title>Restaurant Reservation Availability</title> <meta name="robots" content="noindex" > </meta> <link rel="shortcut icon" href="//components.otstatic.com/components/favicon/1.0.4/favicon/favicon.ico" type="image/x-icon"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.4/favicon/favicon-16.png" sizes="16x16"/><link rel='

' <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/> <title>Restaurant Reservation Availability</title> <meta name="robots" content="noindex" > </meta> <link rel="shortcut icon" href="//components.otstatic.com/components/favicon/1.0.4/favicon/favicon.ico" type="image/x-icon"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.4/favicon/favicon-16.png" sizes="16x16"/><link rel='

In [3]:
# parse raw html => soup object
soup = BeautifulSoup(html.text, 'html.parser')
# for each span tag, print out text => restaurant name
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    print(entry.text)

Quae Bosco
232 Hayes
Unde Hill
Repudiandae
Distinctio Koelpin
312 O'Kon
Towne Alley
At Fritsch
Alivia Point
Rohan Lodge
1112 Hoppe
Knolls
Roads
Mount
Christopher Street
Incidunt
Bobbie Littel
Repellendus Lodge
Vel Altenwerth
Street
Fisher
Dolore Flat
Torphy
Et
Natus
Deckow
Keys
Jermaine Hickle
Veniam Dare
Vel Gaylord
Gails
Rhoda Ford
Minima Mayer
Goldner Street
Bode
Meadow
Assumenda Batz
Alden Brooks
1111 Mante
Omnis Feest
Numquam Bruen
Jaydes
Enim Gardens
Hessel Lodge
Agloe Bar & Grill
Aspernatur Keebler
Littel
Feil Falls
Dolores
Vel
Kemmer
Voluptatum
Pouros
Magni Corwin
Volkman Ferry
Quibusdam Thompson
Bogan Fort
Nihil Ways
Facere Rogahn
Consequuntur
Jacobson
Considine
Padberg
Kassulke
Mollitia Nicolas
Borer Square
Nesciunt
Ward
Wiegand Shoal
Omnis
Enim
1146 Wiegand
537 Gorczany
Maias
Jaycee Nienow
Ea Ryan
Rerum Mount
Expedita
Hic
Canyon
Adalbertos
Et Wiegand
Ortiz Union
Hills
Harvey Loop
Roxanne Hintz
1491 Borer
Eius
29 Kuvalis
Jeds
Abdiel Rutherford
Welch Spring
Rapid
Laboriosam Da

In [4]:
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    price = entry.find('i').text

In [5]:
soup.find_all('div', {'class':'booking'})

[]

In [6]:
from selenium import webdriver
# create a driver called Firefox
driver = webdriver.Chrome()

In [7]:
driver.close()

In [8]:
import time, re
# visit the opentable listing page
driver = webdriver.Chrome()
driver.get(url)
time.sleep(1) # wait 1 second
# retrieve the html source
html = driver.page_source
html = BeautifulSoup(html, "lxml")
for booking in html.find_all('div', {'class': 'booking'}):
    match = re.search(r'\d+', booking.text)
    if match:
        print(match.group())

8
5
17
26
105
85
16
9
31
11
25
13
23
35
5
21
38
13
98
13
181
13
23
2
6
12
41
36
19
230
2
13
31
67
52
8
10
10
14
9
20
3
2
3
23
19
27
7
27
20
4
3
16
33
80
9
51
4
127
27
11
16
1
22
25
32
10
46
4
1
5
22
36
6
4
9
15
9


In [13]:
import re
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver


def parse_html(html):
    """Parse content from various tags from OpenTable restaurants listing"""
    data, item = pd.DataFrame(), {}
    soup = BeautifulSoup(html, 'lxml')
    for i, resto in enumerate(soup.find_all('div', class_='rest-row-info')):
        item['name'] = resto.find('span', class_='rest-row-name-text').text

        booking = resto.find('div', class_='booking')
        item['bookings'] = re.search('\d+', booking.text).group() if booking else 'NA'

        rating = resto.find('div', class_='star-rating-score')
        item['rating'] = float(rating['aria-label'].split()[0]) if rating else 'NA'

        reviews = resto.find('span', class_='underline-hover')
        item['reviews'] = int(re.search('\d+', reviews.text).group()) if reviews else 'NA'

        item['price'] = int(resto.find('div', class_='rest-row-pricing').find('i').text.count('$'))
        item['cuisine'] = resto.find('span', class_='rest-row-meta--cuisine rest-row-meta-text sfx1388addContent').text
        item['location'] = resto.find('span', class_='rest-row-meta--location rest-row-meta-text sfx1388addContent').text
        data[i] = pd.Series(item)
    return data.T


# Start selenium and click through pages until reach end
# store results by iteratively appending to csv file
driver = webdriver.Chrome()
url = "https://www.opentable.com/new-york-restaurant-listings"
driver.get(url)
page = collected = 0
while True:
    sleep(1)
    new_data = parse_html(driver.page_source)
    if new_data.empty:
        break
    if page == 0:
        new_data.to_csv('results.csv', index=False)
    elif page > 0:
        new_data.to_csv('results.csv', index=False, header=None, mode='a')
    page += 1
    collected += len(new_data)
    print(f'Page: {page} | Downloaded: {collected}')
    try:
        driver.find_element_by_link_text('Next').click()
    except:
        print(f'Done')
        break

driver.close()
restaurants = pd.read_csv('results.csv')
print(restaurants)

Page: 1 | Downloaded: 100
Page: 2 | Downloaded: 200
Page: 3 | Downloaded: 300
Page: 4 | Downloaded: 400
Page: 5 | Downloaded: 500
Page: 6 | Downloaded: 600
Page: 7 | Downloaded: 700
Page: 8 | Downloaded: 800
Page: 9 | Downloaded: 900
Page: 10 | Downloaded: 993
Done
                                       name  bookings  rating  reviews  price  \
0                                 Bookmarks       2.0     4.8      9.0      3   
1       HAVEN Riverfront Restaurant and Bar      34.0     4.6   2802.0      3   
2               The Leopard at des Artistes      27.0     4.6   2447.0      4   
3                                     Dagon       4.0     4.6    552.0      3   
4                          The Milling Room      13.0     4.4   2106.0      3   
..                                      ...       ...     ...      ...    ...   
988  White Oak Oyster Bar & Cocktail Lounge       NaN     NaN      NaN      2   
989                           Double Dragon       NaN     NaN      NaN      2   
990  

In [17]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
__author__ = 'Stefan Jansen'

import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '\\Users\\HP-Z440\\Anaconda Projects\Machine-Learning-for-Algorithmic-Trading-Second-Edition\\03_alternative_data\\01_opentable')

from opentable.items import OpentableItem
from scrapy import Spider
from scrapy_splash import SplashRequest


class OpenTableSpider(Spider):
    name = 'opentable'
    start_urls = ['https://www.opentable.com/new-york-restaurant-listings']

    def start_requests(self):
        for url in self.start_urls:
            yield SplashRequest(url=url,
                                callback=self.parse,
                                endpoint='render.html',
                                args={'wait': 1},
                                )

    def parse(self, response):
        item = OpentableItem()
        for resto in response.css('div.rest-row-info'):
            item['name'] = resto.css('span.rest-row-name-text::text').extract()
            item['bookings'] = resto.css('div.booking::text').re(r'\d+')
            item['rating'] = resto.css('div.all-stars::attr(style)').re_first('\d+')
            item['reviews'] = resto.css('span.star-rating-text--review-text::text').re_first(r'\d+')
            item['price'] = len(resto.css('div.rest-row-pricing > i::text').re('\$'))
            item['cuisine'] = resto.css('span.rest-row-meta--cuisine::text').extract()
            item['location'] = resto.css('span.rest-row-meta--location::text').extract()
            yield item
